In [97]:
import numpy as np

In [105]:
class Node (object):
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        self.value = None
        
        for n in inbound_nodes:
            n.outbound_nodes.append(self)
        
    def whoamI(self):
        print("I am a Node")
        
    def forward(self):
        raise NotImplemented

In [106]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
    def whoamI():
        print("I am a Input Node")    
    def forward(self):
        pass

In [116]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self,[inputs, weights, bias])
        
    def whoamI(self):
        print("I am a linear Node")

    def forward(self):
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2]
        self.value = np.dot(inputs, weights) + bias.value
            

In [117]:
class Sigmoid(Node):
    def __init__(self, node):
        Node.__init__(self, [node])
    
    def _sigmoid(self, x):
        return 1. / (1. + np.exp(-x))
        
    def forward(self):
        self.inbound_nodes[0].whoamI()
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

In [128]:
class MSE(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y, a])
        
    def forward(self):
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        
        diff = (y - a)
        sq = diff ** 2
        tot = sq.sum()
        self.value = tot / len(y)

In [129]:
def topological_sort(feed_dict):
    input_nodes = [n for n in feed_dict.keys()]
    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
    #print("G.elements",G.items())
    L = []
    S=set(input_nodes)
    while(len(S) > 0):
        n = S.pop()
        
        if isinstance(n, Input):
            n.value = feed_dict[n]
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    #print('L', L)
    return L

In [130]:
def forward_pass(sorted_nodes, ouput_node = None ):
    for n in sorted_nodes:
        n.forward()
    if ouput_node != None:
        return ouput_node.value

In [133]:
def f(x):
    return x**2 + 5

def g(x):
    return 2*x

In [134]:
def gradient_descent_update(x, gradx, lr):
    return x - (gradx * lr)

In [135]:
import random

x = random.randint(0, 10000)
lr = 0.001
epochs = 100

for i in range(epochs+1):
    cost = f(x)
    gradx = g(x)
    print("Epoch {}: Cost = {}, x = {}".format(i, cost, gradx))
    x = gradient_descent_update(x, gradx, lr)

Epoch 0: Cost = 3150630, x = 3550
Epoch 1: Cost = 3138040.1025, x = 3542.9
Epoch 2: Cost = 3125500.5142304106, x = 3535.8142000000003
Epoch 3: Cost = 3113011.0341555458, x = 3528.7425716000002
Epoch 4: Cost = 3100571.46204306, x = 3521.6850864568
Epoch 5: Cost = 3088181.5984607358, x = 3514.6417162838866
Epoch 6: Cost = 3075841.244773287, x = 3507.6124328513188
Epoch 7: Cost = 3063550.2031391724, x = 3500.597207985616
Epoch 8: Cost = 3051308.276507428, x = 3493.5960135696446
Epoch 9: Cost = 3039115.268614504, x = 3486.608821542505
Epoch 10: Cost = 3026970.983981121, x = 3479.6356038994204
Epoch 11: Cost = 3014875.2279091324, x = 3472.6763326916216
Epoch 12: Cost = 3002827.8064784077, x = 3465.7309800262383
Epoch 13: Cost = 2990828.52654372, x = 3458.799518066186
Epoch 14: Cost = 2978877.1957316515, x = 3451.8819190300537
Epoch 15: Cost = 2966973.622437508, x = 3444.9781551919937
Epoch 16: Cost = 2955117.615822247, x = 3438.0881988816095
Epoch 17: Cost = 2943308.9858094216, x = 3431.212

In [131]:
X, W, b = Input(), Input(), Input()

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

li = Linear(X, W, b)
sig = Sigmoid(li)

feed_dict = {X:X_, W:W_, b:b_}

graph = topological_sort(feed_dict)
output = forward_pass(graph, sig)
print("output", output)

I am a linear Node
output [[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


In [132]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(sorted_nodes=graph)

"""
Expected output

23.4166666667
"""
print(cost.value)


23.4166666667
